<a id="introduction"></a>

# PS2: Logistic Regression Classification of a Clinical Dataset
Can simple linear classifiers predict whether a heart-failure patient will survive based on routine clinical measurements? In this problem set, we address that question by training a Perceptron and a Logistic Regression classifier on a real clinical dataset of 299 patients.

We preprocess the data, visualize it with PCA, train both classifiers on the same train/test split, and compare their performance using confusion-matrix metrics. Work through the notebook, correct any issues, and answer the discussion questions at the end.

> **Learning Objectives:**
> 
> By the end of this problem set, you should be able to:
> 
> * **Preprocess clinical data for classification:** Recode binary features to $\{-1,1\}$, apply z-score normalization to continuous features, and split into training and test sets.
> * **Use PCA to visualize high-dimensional patient data:** Compute the covariance matrix, perform eigendecomposition, and project onto the top two principal components to assess class separability.
> * **Train, evaluate, and compare linear classifiers:** Fit perceptron and logistic regression models and assess accuracy, precision, and recall on the test set using confusion matrices.
___

<a id="setup"></a>

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our lab problem.

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl.git`
   Resolving package versions...
   Installed Libmount_jll ───────── v2.41.3+0
   Installed Opus_jll ───────────── v1.6.1+0
   Installed StaticArrayInterface ─ v1.9.0
   Installed Plots ──────────────── v1.41.5
   Installed FileIO ─────────────── v1.18.0
   Installed NearestNeighbors ───── v0.4.27
   Installed LinearOperators ────── v2.12.0
   Installed Libuuid_jll ────────── v2.41.3+0
   Installed Glib_jll ───────────── v2.86.3+0
   Installed ForwardDiff ────────── v1.3.2
   Installed MathOptInterface ───── v1.49.0
  Installing 2 artifacts
   Installed artifact Opus                  1.2 MiB
   Installed artifact Glib                  7.6 MiB
    Updating `C:\Users\evan\OneDrive\Documents\GitHub\ps2-5820-s26-EvoTW-source\Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
  [336ed68f] + CSV v0.10.15
  [c3611d14] + ColorVectorSpace v0.11.0
  [a93c6f00] + DataFrames v1.8.1
  [8bb1440f] + Delimi

<a id="data"></a>

### Data
Next, let's load up the dataset that we will explore. The data for this lab was taken from this `2020` publication:
* [Davide Chicco, Giuseppe Jurman: "Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone." BMC Medical Informatics and Decision Making 20, 16 (2020). https://doi.org/10.1186/s12911-020-1023-5](https://pubmed.ncbi.nlm.nih.gov/32013925/)

In this paper, the authors analyzed a dataset of 299 heart failure patients collected in 2015. The patients comprised 105 women and 194 men, aged between 40 and 95 years old. The dataset contains 13 features (a mixture of continuous and categorical data), which report clinical, body, and lifestyle information:
* Some features are binary: anemia, high blood pressure, diabetes, sex, and smoking status.
* The remaining features were continuous biochemical measurements, such as the level of the Creatinine phosphokinase (CPK) enzyme in the blood, the number of platelets, etc.
* The class (target) variable is encoded as a binary (boolean) death event: `1` if the patient died during the follow-up period, `0` if the patient did not die during the follow-up period.

We'll load this dataset as a [DataFrame instance](https://dataframes.juliadata.org/stable/) and store it in the `originaldataset::DataFrame` variable:

In [2]:
originaldataset = MyHeartDiseaseClinicalDataset(); # load the heart disease dataset

<a id="scaling"></a>

#### Data scaling
Our binary classification models require [a `Matrix`](https://docs.julialang.org/en/v1/base/arrays/#Base.Matrix-Tuple{UndefInitializer,%20Any,%20Any}), not [a `DataFrame`](https://dataframes.juliadata.org/stable/). We preprocess the data in three steps:

> __Data Preprocessing:__
> 
> * __Binary recoding:__ Convert categorical `0,1` data to `-1,1` where `0` maps to `-1` and `1` remains `1`.
> * __Z-score normalization:__ Apply [z-score scaling](https://en.wikipedia.org/wiki/Feature_scaling) to continuous features using $x^{\prime} = (x - \mu)/\sigma$ where $\mu$ is the mean and $\sigma$ is the standard deviation.
> * __Label retention:__ Keep the `death_event` label because the classifiers are supervised.

The preprocessed feature matrix is stored in `X::Matrix{Float64}`, while the label vector is stored in `y::Vector{Float64}`. We also keep the treated dataset in `dataset::DataFrame`.

In [3]:
(X, y, dataset) = let

    # TODO: Implement data preprocessing and scaling.
    # Steps:
    # 1) Copy `originaldataset` to `treated_dataset`.
    # 2) Map binary columns to -1/1: :anaemia, :diabetes, :high_blood_pressure, :sex, :smoking, :death_event.
    # 3) Build data matrix `D` from `treated_dataset`.
    # 4) Z-scale the continuous columns listed in `index_to_z_scale`.
    # 5) Split features (X) and labels (y), then return X, y, treated_dataset.
    # NOTE: Replace the placeholder assignments below with your implementation.

    # initialize outputs -
    X = nothing # features matrix
    y = nothing # labels vector
    treated_dataset = nothing # updated dataset after preprocessing

    index_to_z_scale = [
        1;  # age
        3;  # creatinine_phosphokinase
        5;  # ejection_fraction
        7;  # platelets
        8;  # serum_creatinine
        9;  # serum_sodium
        12; # time
    ];

    #=
    # Original solution (commented out). Use this block only as a reference once you've attempted the task.
    treated_dataset = copy(originaldataset);
    transform!(treated_dataset, :anaemia => ByRow(x -> (x==0 ? -1 : 1)) => :anaemia);
    transform!(treated_dataset, :diabetes => ByRow(x -> (x==0 ? -1 : 1)) => :diabetes);
    transform!(treated_dataset, :high_blood_pressure => ByRow(x -> (x==0 ? -1 : 1)) => :high_blood_pressure);
    transform!(treated_dataset, :sex => ByRow(x -> (x==0 ? -1 : 1)) => :sex);
    transform!(treated_dataset, :smoking => ByRow(x -> (x==0 ? -1 : 1)) => :smoking);
    transform!(treated_dataset, :death_event => ByRow(x -> (x==0 ? -1 : 1)) => :death_event);

    D = treated_dataset[:,1:end] |> Matrix;
    (number_of_examples, number_of_features) = size(D);

    D̂ = copy(D);
    for i ∈ eachindex(index_to_z_scale)
        j = index_to_z_scale[i];
        μ = mean(D[:,j]);
        σ = std(D[:,j]);
        for k ∈ 1:number_of_examples
            D̂[k,j] = (D[k,j] - μ)/σ;
        end
    end

    X = D̂[:,1:end-1]; # features matrix 
    y = D̂[:,end]; # last column is the labels vector
    =#

    X, y, treated_dataset
end;

Now, let's form the centered data matrix $\tilde{\mathbf{X}}$ by subtracting the mean from each column (feature) of the data matrix $\mathbf{X}$. We store the centered data in the `X̃::Array{Float64,2}` variable:

In [4]:
X̃ = let
    # TODO: Center X by subtracting the mean of each column.
    # HINT: r, c = size(X); m = mean(X, dims=1) |> vec; X̃ = X .- ⊗(ones(r), m)
    # NOTE: Replace the placeholder assignment below with your implementation.

    # initialize outputs -
    X̃ = nothing; # centered data matrix

    #=
    r, c = size(X)
    m = mean(X, dims=1) |> vec
    ones_vector = ones(r)
    X̃ = X .- ⊗(ones_vector, m); # outer product replicates the mean vector
    =#

    X̃ # return centered data matrix
end


Finally, let's compute the empirical covariance matrix $\hat{\mathbf{\Sigma}}$ and store it in the `Σ̂::Array{Float64,2}` variable:

In [5]:
Σ̂ = let

    # TODO: Compute the empirical covariance matrix of X̃.
    # HINT: Σ = (1/(r-1)) * (transpose(X̃) * X̃)
    # NOTE: Replace the placeholder assignment below with your implementation.

    # initialize outputs -
    Σ = nothing; # empirical covariance matrix

    #=
    (r,c) = size(X̃)
    Σ = (1/(r-1)) * (transpose(X̃) * X̃); # empirical covariance matrix
    =#

    Σ; # return the empirical covariance matrix
end

Next, split the centered dataset `X̃` into `training::NamedTuple` and `test::NamedTuple` subsets using a random 80/20 partition. Each named tuple has fields `X::Array{Float64,2}` (the feature matrix with a bias column appended) and `y::Vector{Float64}` (the labels). The `training` data will be used to estimate model parameters and the `test` data for evaluation.

In [6]:
training, test = let

    # TODO: Create an 80/20 train-test split with randomized indices.
    # Build tuples: training = (X=..., y=...), testing = (X=..., y=...).
    # HINT: include a bias column of ones in both training.X and testing.X.
    # NOTE: Replace the placeholder assignments below with your implementation.

    # initialize outputs -
    training = nothing; # named tuple for training data
    testing = nothing; # named tuple for testing data

    #=
    s = 0.80; # fraction of data for training
    D = X̃; # use scaled features
    number_of_training_samples = Int(round(s * size(D,1)));
    i = randperm(size(D,1));
    training_indices = i[1:number_of_training_samples];
    testing_indices = i[number_of_training_samples+1:end];

    one_vector = ones(number_of_training_samples);
    training = (X=[D[training_indices, :] one_vector], y=y[training_indices]);

    one_vector = ones(length(testing_indices));
    testing = (X=[D[testing_indices, :] one_vector], y=y[testing_indices]);
    =#

    training, testing
end;

Do some checks. 

In [7]:
let
    println("Sanity checks (data shapes and labels):")
    println("X size: ", size(X), ", y length: ", length(y))
    println("X̃ size: ", size(X̃))
    println("Training X size: ", size(training.X), ", Training y length: ", length(training.y))
    println("Test X size: ", size(test.X), ", Test y length: ", length(test.y))

    label_values = sort(unique(y))
    println("Label values: ", label_values)
    if length(label_values) != 2 || !(all(v -> v in (-1.0, 1.0), label_values))
        println("Warning: unexpected label values; expected -1 and 1.")
    end

    center_error = maximum(abs.(mean(X̃, dims=1)))
    println("Centering check (max abs mean of X̃): ", center_error)
    if center_error > 1e-8
        println("Warning: centered features are not near zero mean.")
    end

    if any(isnan, X) || any(isnan, y)
        println("Warning: NaNs detected in X or y.")
    end
    if any(isinf, X)
        println("Warning: Infs detected in X.")
    end

    nothing
end

Sanity checks (data shapes and labels):


MethodError: MethodError: no method matching size(::Nothing)
The function `size` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  size(!Matched::CSV.ReversedBuf)
   @ CSV C:\Users\evan\.julia\packages\CSV\XLcqT\src\utils.jl:569
  size(!Matched::Compiler.TwoPhaseVectorView)
   @ Base C:\Users\evan\.julia\juliaup\julia-1.12.4+0.x64.w64.mingw32\share\julia\Compiler\src\inferencestate.jl:81
  size(!Matched::PrettyTables.ColumnTable)
   @ PrettyTables C:\Users\evan\.julia\packages\PrettyTables\ePffU\src\tables.jl:75
  ...


Finally, set up the color dictionary to visualize the classification datasets. The keys of the `my_color_dictionary::Dict{Int64, RGB}` dictionary are class labels, i.e., $ y\in\{1,-1\}$, while the values are the colors mapped to that label.

In [8]:
my_color_dictionary = Dict{Int64,RGB}();
my_color_dictionary[1] = colorant"#03045e"; # color for Label = 1 (you can change color if you want!)
my_color_dictionary[-1] = colorant"#e36414"; # color for Label = -1 (you can change color if you want!)

___

<a id="task1"></a>

## Task 1: Visualize the dataset using Principal Component Analysis (PCA)
In this task, you will reduce the dimension of the scaled dataset so that we can plot it in two dimensions.
In general, imagine that we have a set of $m$-dimensional features $\mathbf{x}\in\mathbb{R}^{m}$ that we want to reduce to a new set of composite feature vectors with a smaller dimension $\mathbf{y}\in\mathbb{R}^{k}$ where $k\ll{m}$. In our case, we have a `12` feature dimension, but we want to project this into `2` dimensions to visualize the label pattern.

Suppose we have a transformation matrix $\mathbf{P}\in\mathbb{R}^{k\times{m}}$ so that: $\mathbf{y} = \mathbf{P}\;(\mathbf{x} - \bar{\mathbf{x}})$ where $\mathbf{y}\in\mathbb{R}^{k}$ is the new composite feature vector, $\mathbf{x}\in\mathbb{R}^m$ is the original feature vector, and $\bar{\mathbf{x}}$ is the mean of the features in the (original) data.  

> **Why center the data?** We subtract the mean $\bar{\mathbf{x}}$ because dimensionality reduction seeks directions of maximum variance in the data. If we do not center, we measure distance from the origin rather than spread around the data center. Centering ensures the reduced representation captures variation in the data.

If we write $\mathbf{P} = [\,\mathbf{\phi}_1^\top;\dots;\mathbf{\phi}_k^\top]$, then each row $\mathbf{\phi}_i^\top$ extracts one component of the new composite feature vector $\mathbf{y}$:
$$
\begin{align*}
y_{i} = \mathbf{\phi}_{i}^{\top}\;(\mathbf{x} - \bar{\mathbf{x}})\quad{i=1,2,\dots,k}\quad\forall{\mathbf{x}\in\mathcal{D}}
\end{align*}
$$

What are these transformation vectors $\mathbf{\phi}_{i}^{\top}$? 

In brief, the $\mathbf{\phi}_{i}^{\top}$ vectors are the top-$k$ eigenvectors (those corresponding to the $k$ largest eigenvalues) of the data's covariance matrix, and this reduction procedure is known as Principal Component Analysis (PCA).

Compute the eigendecomposition of the covariance matrix $\hat{\mathbf{\Sigma}}$ using [the `eigen(...)` method exported by the `LinearAlgebra.jl` package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/) and save the $k\times m$ transformation matrix in `P::Array{Float64,2}`:

In [9]:
P = let

    # TODO: Compute the PCA transformation matrix.
    # Steps:
    # 1) F = eigen(Σ̂)
    # 2) λ = F.values, V = F.vectors
    # 3) Sort eigenpairs by descending λ
    # 4) Keep k = 2 principal components and form P
    # 5) Return PT = transpose(P) as a Matrix
    # NOTE: Replace the placeholder assignments below with your implementation.

    # initialize outputs -
    F = nothing # TODO: eigendecomposition of Σ̂
    λ = nothing # TODO: eigenvalues (sorted)
    V = nothing # TODO: eigenvectors (sorted to match λ)
    P = nothing # TODO: transformation matrix (top k eigenvectors)
    PT = nothing # TODO: transpose(P) |> Matrix

    #=
    F = eigen(Σ̂); # eigendecomposition of the covariance matrix
    λ = F.values;
    V = F.vectors;

    p = sortperm(λ, rev=true); # indices that would sort λ in descending order
    λ = λ[p]
    V = V[:,p]

    k = 2; # number of principal components to keep
    P = V[:, 1:k]; # transformation matrix with top k eigenvectors as columns
    PT = transpose(P) |> Matrix
    =#

    PT
end;

### Visualize
Let's visualize the principal components of the data by projecting the centered data onto the eigenvectors corresponding to the largest eigenvalues. The projected data points will be stored in the `Y::Array{Float64,2}` variable, where $\mathbf{Y} = \tilde{\mathbf{X}}\mathbf{P}^{\top}$ and $\tilde{\mathbf{X}}$ is the centered data matrix.

In [10]:
Y = nothing # TODO: compute Y = (X̃ * transpose(P)) |> Matrix

> __What do we expect to see?__ 
> 
> Since PCA aims to capture the directions of maximum variance in the data, we expect that the first two principal components will reveal some structure in the data. 
> 
> If the features are informative with respect to the `death_event` label, we might observe some clustering or separation between the points corresponding to different classes (i.e., patients who died vs. those who survived).

So what do we see?

In [11]:
# TODO: Create a scatter plot of the first two PCA components in Y.
# - Use `death_event` labels to color points.
# - Label axes and adjust background styling.

#=
# Original solution (commented out)
let
    death_event = dataset[:, :death_event] |> Vector; # extract death_event labels
    scatter([Y[death_event .== 1, 1]], [Y[death_event .== 1, 2]]; color=:red, label="Death Event", markersize=5)
    scatter!([Y[death_event .== -1, 1]], [Y[death_event .== -1, 2]]; color=:navy, label="No Death Event", markersize=5)
    plot!(bg="gray95", background_color_outside="white", framestyle = :box, fg_legend = :transparent);
    xlabel!("Composite Feature 1", fontsize=18)
    ylabel!("Composite Feature 2", fontsize=18)
end
=#

In [12]:
do_I_see_the_PCA_plot = false; # TODO: update this flag {true | false}


> __Feature importance in PCA:__ 
> 
> The eigenvector coefficients indicate how much each of the original $m$ features contributes to a composite feature. For some eigenvector $\mathbf{z}$, the ith scaled component $l(\mathbf{z})_{i}$ is given by:
> $$
\begin{align*}
    l(\mathbf{z})_{i} &= \frac{\text{abs}(z_{i})}{\sum_{j=1}^{m}\text{abs}(z_{j})}\quad{i=1,2,\dots,m}
\end{align*}    
$$
> The scaled vectors should sum to `1`; thus, we can think about the elements (loosely) as probabilities, i.e., the probability that the ith component is the most important. 


Compute the scaled loadings for the first principal component.


In [13]:
# TODO: Compute and display the scaled loadings for the first principal component.
# Build a DataFrame with Feature, Scaled_Loading, and Rank.

#=
# Original solution (commented out)
let
    k = 1; # look at only the first principal component
    ϕ = P[k,:]; # get the transformation vector
    df = DataFrame();

    numerator = abs.(ϕ);
    denominator = sum(numerator);
    scaled_loadings = numerator ./ denominator;

    feature_names = names(dataset)[1:end-1]; # exclude death_event column

    p = sortperm(scaled_loadings, rev=true)
    r = similar(p)
    r[p] = 1:length(scaled_loadings)

    for i ∈ eachindex(feature_names)
        feature = feature_names[i];
        loading = scaled_loadings[i];
        rank = r[i];
        push!(df, (Feature=feature, Scaled_Loading=loading, Rank=rank));
    end

    pretty_table(
        df;
        backend = :text,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end
=#


Next, we move to the perceptron classifier.

___

<a id="task2"></a>

## Task 2: Perceptron Classification using online learning
In this task, we build and train a [Perceptron](https://en.wikipedia.org/wiki/Perceptron) classifier using the training data, and then challenge this classifier using the `test` dataset. 

> __Convergence note:__ If the dataset $\mathcal{D}$ is linearly separable, the Perceptron converges to a separating hyperplane in finite iterations. If $\mathcal{D}$ is not linearly separable, the Perceptron may not converge. To avoid infinite loops, we set a maximum number of mistakes $M$ (e.g., $M=1$) and a maximum number of iterations $T$.

__Initialize__: Given a linearly separable dataset $\mathcal{D} = \left\{(\mathbf{x}_{1},y_{1}),\dotsc,(\mathbf{x}_{n},y_{n})\right\}$, the maximum number of iterations $T$, and the maximum number of mistakes $M$ (e.g., $M=1$), initialize the parameter vector $\mathbf{\theta} = \left(\mathbf{w}, b\right)$ to small random values and set the loop counter $t\gets{0}$.

> **Rule of thumb for maximum iterations**: Set the maximum number of iterations $T = 10n$ to $100n$, where $n$ is the number of training examples. The algorithm often converges faster for linearly separable data. However, for non-separable data, a larger $T$ may be necessary to achieve satisfactory performance.

While $\texttt{true}$ __do__:
1. Initialize the number of mistakes $\texttt{mistakes} = 0$.
2. For each training example $(\mathbf{x}, y) \in \mathcal{D}$: compute $y\;\left(\mathbf{\theta}^{\top}\;\mathbf{x}\right)\leq{0}$. 
    - If $\texttt{true}$: the example is __misclassified__ (the sign of the prediction doesn't match the label $y$). Update $\mathbf{\theta} \gets \mathbf{\theta} + y\;\mathbf{x}$ and increment $\texttt{mistakes} \gets \texttt{mistakes} + 1$.
3. After processing all examples, if $\texttt{mistakes} \leq {M}$ or $t \geq T$, exit. Otherwise, increment $t \gets t + 1$ and repeat from step 1.

We aim to minimize mistakes, with $M = 0$ being ideal. However, zero mistakes may not be achievable for weakly separable or non-separable data.

__Training__: Our Perceptron implementation stores problem data in a [MyPerceptronClassificationModel instance](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/#VLDataScienceMachineLearningPackage.MyPerceptronClassificationModel). We learn parameters using the [learn(...) method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/binaryclassification/#VLDataScienceMachineLearningPackage.learn), which takes the feature array `X`, labels vector `y`, and problem instance, returning an updated instance with the learned parameters. The trained classifier is stored in `model_perceptron::MyPerceptronClassificationModel`.

Train the perceptron model on the training set.


In [14]:
model_perceptron = let

    # TODO: Train a perceptron on the training set.
    # HINT: build(MyPerceptronClassificationModel, (...)), then learn(...).
    # NOTE: Replace the placeholder assignment below with your implementation.

    # initialize outputs -
    model = nothing; # trained perceptron model

    #=
    X = training.X;
    y = training.y;
    number_of_examples = size(X,1);
    number_of_features = size(X,2);
    maxiter = 100*number_of_examples;

    # build initial model
    model = build(MyPerceptronClassificationModel, (
        parameters = ones(number_of_features),
        mistakes = 0
    ));

    # train the model
    model = learn(X,y,model, maxiter = maxiter, verbose = true);
    =#

    model; # return the trained model
end;

Now we evaluate the trained model on unseen test data.

> __Inference__ 
> 
> We run classification on test data using the [classify(...) method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/binaryclassification/#VLDataScienceMachineLearningPackage.classify). This takes the feature array `X` and trained model, returning estimated labels. We store actual labels in `y_perceptron::Vector{Float64}` and predicted labels in `ŷ_perceptron::Vector{Float64}`.

Run inference on the test set to obtain predicted labels.

In [15]:
ŷ_perceptron,y_perceptron = let

    # TODO: Use the trained perceptron to classify the test set.
    # NOTE: Replace the placeholder assignments below with your implementation.

    # initialize outputs -
    ŷ = nothing; # predicted label
    y = nothing; # actual label

    #=
    X = test.X;
    y = test.y;
    ŷ = classify(X,model_perceptron);
    =#

    ŷ, y # return
end;

Before computing the full confusion matrix, we tally the raw number of misclassified test examples in `number_of_prediction_mistakes::Int64` and print the overall mistake percentage.

In [16]:
number_of_prediction_mistakes = let

    # TODO: Count prediction mistakes on the test set.
    # NOTE: Replace the placeholder assignment below with your implementation.

    # initialize outputs -
    error_counter = nothing; # how many mistakes made?

    #=
    number_of_test_examples = length(ŷ_perceptron);
    error_counter = 0;

    for i ∈ 1:number_of_test_examples
        if (ŷ_perceptron[i] != y_perceptron[i])
            error_counter += 1;
        end
    end

    println("Number of Perceptron mistakes: $(error_counter) of $(length(ŷ_perceptron)) test examples.")
    =#

    error_counter; # return the number of mistakes
end;

In [17]:
# TODO: Print the perceptron mistake percentage after computing `number_of_prediction_mistakes`.
#=
# println("Perceptron mistake percentage: $((number_of_prediction_mistakes/length(ŷ_perceptron))*100)%")
=#

Beyond the raw mistake count, we evaluate classifier performance using [the confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix). The confusion matrix compares predicted labels $\hat{y}_{i}$ to actual labels $y_{i}$ and categorizes each prediction as a true positive, true negative, false positive, or false negative.

> **Error analysis using the confusion matrix**
>
> Total mistakes alone do not reveal *where* the classifier fails. In a clinical setting, we need to distinguish: How often did we predict death when the patient survived (false positive)? How often did we predict survival when the patient died (false negative)?
>
> The confusion matrix for a binary classifier:
>
>|                     | **Predicted Positive** | **Predicted Negative** |
>|---------------------|------------------------|------------------------|
>| **Actual Positive** | True Positive (TP)     | False Negative (FN)    |
>| **Actual Negative** | False Positive (FP)    | True Negative (TN)     |
>
> From these counts we derive three metrics:
>
> * **Accuracy** is the fraction of correct predictions: $\texttt{accuracy} = \frac{TP + TN}{TP + TN + FP + FN}$. It can mislead on imbalanced data; a classifier that always predicts "survived" could score high if most patients survived.
>
> * **Precision** answers "when we predict death, how often are we right?" $\texttt{precision} = \frac{TP}{TP + FP}$. High precision means fewer false alarms.
>
> * **Recall** (sensitivity) answers "of all patients who died, how many did we catch?" $\texttt{recall} = \frac{TP}{TP + FN}$. In diagnosis, recall is often more critical; missing a death is worse than a false alarm.

Compute the confusion matrix and store it in `CM_perceptron::Array{Int64,2}`, then extract `accuracy_perceptron::Float64`, `precision_perceptron::Float64`, and `recall_perceptron::Float64`.

In [18]:
CM_perceptron = nothing # TODO: confusion(y_perceptron, ŷ_perceptron) (actual labels first)

In [19]:
accuracy_perceptron, precision_perceptron, recall_perceptron = let

    # TODO: Compute accuracy, precision, and recall from CM_perceptron.
    # NOTE: Replace the placeholder assignments below with your implementation.

    # initialize outputs -
    accuracy = nothing; # accuracy for perceptron
    precision = nothing; # precision for perceptron
    recall = nothing; # recall for perceptron

    #=
    TP = CM_perceptron[1,1];
    TN = CM_perceptron[2,2];
    FP = CM_perceptron[2,1];
    FN = CM_perceptron[1,2];

    accuracy = (TP + TN) / (TP + TN + FP + FN);
    precision = TP / (TP + FP);
    recall = TP / (TP + FN);

    println("Perceptron Accuracy: $(round(accuracy*100,digits=2))%")
    println("Perceptron Precision: $(round(precision*100,digits=2))%")
    println("Perceptron Recall: $(round(recall*100,digits=2))%")

    =#

    accuracy, precision, recall; # return
end;

___

<a id="task3"></a>

## Task 3: Logistic Regression using Gradient Descent
In this task, we build and train a [Logistic regression](https://en.wikipedia.org/wiki/Logistic_regression) classifier using the training data, and then challenge this classifier using the `test` dataset. We'll use [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) to minimize the negative log-likelihood loss function. A [pseudocode outline of our training algorithm can be found here](docs/CHEME-5820-Algorithm-Simplified-GD-Spring-2026.ipynb).

We implemented [the `MyLogisticRegressionClassificationModel` type](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/), which contains data required to solve the logistic regression problem, i.e., parameters, the learning rate, a stopping tolerance parameter $\epsilon$, and a loss (objective) function that we want to minimize.

Convergence uses the tolerance $\epsilon$ on parameter change: stop when $\|\theta^{(t)}-\theta^{(t-1)}\|_{2} \le \epsilon$.

> __Details__
> 
> * __Technical note__: In this implementation, we approximate the gradient using a forward finite difference.
> * __Note on the loss function__: In the code below, we use the natural logarithm `log` in the loss function. You could also use `log10`. While this differs from the mathematical derivation above (which uses natural log), it does not change the location of the minimum since `log10` is simply a scaled version of the natural log. The gradient descent algorithm will find the same optimal parameters $\theta$.
> * In the code block below, we [build a `model::MyLogisticRegressionClassificationModel` instance using a `build(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/). The model instance initially has a random guess for the classifier parameters. We use gradient descent to refine that guess [using the `learn(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/binaryclassification/), which returns an updated model instance (with the best parameters that we found so far). 

We return the updated model instance and save it in the `model_logistic_gd::MyLogisticRegressionClassificationModel` variable.


In [20]:
model_logistic_gd = let

    # TODO: Train logistic regression via gradient descent.
    # HINT: build(MyLogisticRegressionClassificationModel, (...)), then learn(...).
    # NOTE: Replace the placeholder assignment below with your implementation.

    # initialize outputs -
    model = nothing; # trained logistic regression model

    #=
    X = training.X;
    y = training.y;
    number_of_features = size(X,2);
    T = 1.0;
    h = 1e-6;
    λ = 0.0;
    ϵ = 1e-6;

    model = build(MyLogisticRegressionClassificationModel, (
        parameters = 0.01*ones(number_of_features),
        learning_rate = 0.005,
        ϵ = ϵ,
        h = h,
        λ = λ,
        T = T,
        loss_function = (x,y,T,λ,θ) -> log(1+exp(-2*y*T*(dot(x,θ)))) + λ*norm(θ,2)^2
    ));

    # train the model
    model = learn(X,y,model, maxiter = 20000, verbose = true);
    =#

    model; # return the trained model
end;

Now we test how well the trained `model_logistic_gd::MyLogisticRegressionClassificationModel` instance classifies data it has never seen, i.e., the `test` dataset.

> __Inference__: We run classification on the test data [using the `classify(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/binaryclassification/). Unlike the Perceptron, logistic regression returns class probabilities: each row of the output matrix is a test instance and each column corresponds to a label (`1` or `-1`). We convert these to hard predictions by selecting the highest-probability class.

We store the predicted labels in `ŷ_logistic_gd::Vector{Float64}`, the actual labels in `y_logistic_gd::Vector{Float64}`, and the raw class probabilities in `probability_matrix::Array{Float64,2}`.

In [21]:
ŷ_logistic_gd,y_logistic_gd, probability_matrix = let

    # TODO: Use logistic regression to classify the test set.
    # HINT: classify(...) returns an n x 2 probability matrix.
    # NOTE: Replace the placeholder assignments below with your implementation.

    # initialize outputs -
    ŷ = nothing; # predicted labels
    y = nothing; # actual labels
    P = nothing; # probability matrix

    #=
    D = test.X;
    y = test.y;
    number_of_examples = size(D,1);

    P = classify(D, model_logistic_gd)

    ŷ = zeros(number_of_examples);
    for i ∈ 1:number_of_examples
        a = argmax(P[i,:]);
        ŷ[i] = 1;
        if (a == 2)
            ŷ[i] = -1;
        end
    end
    =#

    ŷ, y, P; # return
end;

We evaluate the logistic regression classifier using the same confusion-matrix framework introduced in Task 2. Although logistic regression outputs class probabilities rather than hard labels, we converted these to predictions in the previous cell by selecting the highest-probability class.

Compute the confusion matrix and store it in `CM_logistic_gd::Array{Int64,2}`, then extract `accuracy_logistic::Float64`, `precision_logistic::Float64`, and `recall_logistic::Float64`.

In [22]:
CM_logistic_gd = nothing # TODO: confusion(y_logistic_gd, ŷ_logistic_gd) (actual, predicted)

In [23]:
accuracy_logistic, precision_logistic, recall_logistic = let

    # TODO: Compute accuracy, precision, and recall from CM_logistic_gd.
    # NOTE: Replace the placeholder assignments below with your implementation.

    # initialize outputs -
    accuracy = nothing; # accuracy for logistic regression
    precision = nothing; # precision for logistic regression
    recall = nothing; # recall for logistic regression

    #=
    TP = CM_logistic_gd[1,1];
    TN = CM_logistic_gd[2,2];
    FP = CM_logistic_gd[2,1];
    FN = CM_logistic_gd[1,2];

    accuracy = (TP + TN) / (TP + TN + FP + FN);
    precision = TP / (TP + FP);
    recall = TP / (TP + FN);

    println("Logistic Regression Accuracy: $(round(accuracy*100,digits=2))%")
    println("Logistic Regression Precision: $(round(precision*100,digits=2))%")
    println("Logistic Regression Recall: $(round(recall*100,digits=2))%")

    =#

    accuracy, precision, recall; # return
end;

What does `probability_matrix::Array{Float64,2}` look like? If the classification is confident, probabilities will be close to 0 or 1; if uncertain, they will cluster near 0.5. The first column corresponds to the probability of class `1` and the second column to class `-1`.

In [24]:
probability_matrix # TODO: display probability_matrix after computing logistic regression

___

<a id="discussion"></a>

## Discussion
We now have two trained classifiers for the same clinical prediction task. The tables below compare their confusion-matrix entries and learned parameters side by side. Use these comparisons to answer the discussion questions that follow.

In [25]:
# TODO: Build a comparison table for Logistic Regression (GD) and Perceptron.
# Include TP, FN, FP, TN, Accuracy, Precision, Recall.

#=
# Original solution (commented out)
let
    df = DataFrame();

    push!(df, (
        label = "Logistic Regression (GD)",
        TP = CM_logistic_gd[1,1],
        FN = CM_logistic_gd[1,2],
        FP = CM_logistic_gd[2,1],
        TN = CM_logistic_gd[2,2],
        Accuracy = round(accuracy_logistic, digits=4),
        Precision = round(precision_logistic, digits=4),
        Recall = round(recall_logistic, digits=4)
    ))

    push!(df, (
        label = "Perceptron",
        TP = CM_perceptron[1,1],
        FN = CM_perceptron[1,2],
        FP = CM_perceptron[2,1],
        TN = CM_perceptron[2,2],
        Accuracy = round(accuracy_perceptron, digits=4),
        Precision = round(precision_perceptron, digits=4),
        Recall = round(recall_perceptron, digits=4)
    ))

    pretty_table(
        df;
        backend = :text,
        fit_table_in_display_horizontally = false,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end
=#


In [26]:
do_I_see_confusion_table = false; # TODO: update this flag {true | false}

Next, compare the parameter values estimated by the perceptron and logistic regression models. The `abs_p_diff` field denotes the absolute percentage difference from the perceptron parameters.

In [27]:
# TODO: Build a parameter comparison table between perceptron and logistic regression.

#=
# Original solution (commented out)
let
    df = DataFrame();
    θ₁ = model_perceptron.β;
    θ₂ = model_logistic_gd.β;
    number_of_parameters = length(θ₁);

    for i ∈ 1:number_of_parameters
        row_df = (
            parameter_index = i,
            Perceptron = θ₁[i],
            Logistic = θ₂[i],
            abs_p_diff = 100*abs((θ₁[i] - θ₂[i])/θ₁[i])
        );
        push!(df, row_df);
    end

    pretty_table(
        df;
        backend = :text,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end
=#


In [28]:
do_I_see_parameter_table = false; # TODO: update this flag {true | false}

**DQ1: Which classifier would you deploy for mortality screening?** The comparison table above shows confusion-matrix entries and metrics for both models on the same test split. In a clinical setting, missing a patient who will die (false negative) and flagging a healthy patient for urgent follow-up (false positive) carry very different costs.

> __Strategy__: Using the accuracy, precision, and recall values from the table, state which classifier you would choose for this dataset. Justify your choice in 2–3 sentences, explicitly addressing whether you prioritize precision or recall for a mortality screening task and why.

In [29]:
# Answer DQ1 here after running the models and recording metrics.

In [30]:
did_I_answer_DQ1 = false; # TODO: update to true if you answered DQ1 {true | false}

**DQ2: How does the inverse temperature $\beta$ affect clinical predictions?** In the logistic regression loss function, the inverse temperature (called `T` in the code) controls how sharply the model separates classes. A small `T` produces soft, uncertain probabilities; a large `T` pushes probabilities toward 0 or 1.

> __Strategy:__ Change `T` in the logistic regression training cell to at least three values (e.g., $2^{-2}$, $2^{0}$, $2^{2}$), re-run the training and inference cells each time, and record the accuracy, precision, and recall. How do these metrics shift as `T` increases? Which value of `T` would you choose for this clinical task, and does the answer depend on whether you prioritize catching all deaths (recall) versus avoiding false alarms (precision)? Explain in 2–3 sentences.

In [31]:
# Answer DQ2 here after sweeping `T` and recording metrics.

In [32]:
did_I_answer_DQ2 = false; # TODO: update to true if you answered DQ2 {true | false}

**DQ3: How does regularization $\lambda$ affect the learned weights and test performance?** The L2 regularization term $\lambda\|\boldsymbol{\theta}\|_{2}^{2}$ penalizes large weights, trading off training-set fit against generalization. Over-regularizing can force the model to ignore risk factors that genuinely predict mortality.

> __Strategy:__ Reset `T` to `1.0`, then change `λ` in the logistic regression training cell to at least three values (e.g., $0$, $10^{-2}$, $10^{-1}$). Re-run training and check both the test metrics and the parameter comparison table. How do the weight magnitudes and classification metrics change as $\lambda$ increases? At what point does regularization start to hurt test performance, and what does that tell you about the complexity needed for this dataset? Explain in 2–3 sentences.

In [33]:
# Answer DQ3 here after sweeping `\lambda` and recording metrics.

In [34]:
did_I_answer_DQ3 = false; # TODO: update to true if you answered DQ3 {true | false}

**DQ4: Do PCA, the perceptron, and logistic regression agree on which clinical features matter?** The PCA loadings table in Task 1 ranks features by their contribution to the first principal component. The parameter comparison table above shows the learned weights for both classifiers. Different methods can emphasize different features depending on their objective; PCA maximizes variance, while the classifiers minimize prediction error.

> __Strategy:__ Compare the top-3 features identified by PCA loadings, the perceptron weights, and the logistic regression weights (use absolute values for a fair comparison). Where do the rankings agree, and where do they diverge? [Chicco and Jurman (2020)](https://pubmed.ncbi.nlm.nih.gov/32013925/) found that serum creatinine and ejection fraction alone could predict survival; do your three methods support that finding? Discuss in 3–4 sentences.

In [35]:
# Answer DQ4 here after computing feature rankings across methods.

In [36]:
did_I_answer_DQ4 = false; # TODO: update to true if you answered DQ4 {true | false}

___

<a id="summary"></a>

## Summary
We investigated whether routine clinical risk factors can predict heart-failure mortality using two linear classifiers. After preprocessing the 299-patient dataset (binary recoding and z-score normalization), PCA visualization revealed the degree of class overlap in two dimensions, and both the perceptron and logistic regression were trained and evaluated on the same 80/20 split.

> __Key Takeaways:__
>
> * **Preprocessing determines model input quality:** Recoding binary features to $\{-1,1\}$ and z-score scaling continuous features places all inputs on comparable scales for gradient-based learning.
> * **PCA reveals class separability before training:** Projecting onto the top two eigenvectors provides a visual check of whether the classes are likely separable by a linear boundary.
> * **Confusion-matrix metrics expose different failure modes:** Accuracy alone can hide class imbalance; precision and recall reveal whether the classifier is biased toward false positives or false negatives, a critical distinction for clinical deployment.

This exercise illustrates the end-to-end process of building and evaluating linear classifiers on a real clinical dataset, highlighting the importance of preprocessing, visualization, and nuanced performance metrics in machine learning for healthcare.

___

<a id="tests"></a>

## Tests
In the code block below, we check some values in your notebook and give you feedback on which items are correct or different. `Unhide` the code block below (if you are curious) about how we implemented the tests and what we are testing.

In [37]:
let 
    @testset verbose = true "CHEME 5820 problem set 2 test suite" begin
        
        @testset "Setup, Prerequisites and Data" begin
            @test _DID_INCLUDE_FILE_GET_CALLED == true
            @test isnothing(X) == false
            @test isnothing(y) == false
            @test isnothing(dataset) == false
            @test isempty(Σ̂) == false
            @test size(X,1) == length(y)
            @test size(dataset,1) == size(X,1)
            @test all(v -> v in (-1.0, 1.0), y)
        end

        @testset "Task 1: PCA" begin
            @test isnothing(P) == false
            @test isnothing(Y) == false
            @test size(Y,1) == size(X̃,1)
            @test size(Y,2) == 2
            @test size(P,1) == 2
            @test size(P,2) == size(X,2)
            @test maximum(abs.(mean(X̃, dims=1))) < 1e-8
            @test isapprox(Σ̂, transpose(Σ̂), atol=1e-8)
            @test minimum(eigvals(Σ̂)) >= -1e-8
            @test isapprox(P * transpose(P), I(2), atol=1e-6)
            @test isapprox(Y, X̃ * transpose(P), atol=1e-8)
            @test do_I_see_the_PCA_plot == true
        end

        @testset "Task 2: Logistic Regression using Gradient Descent" begin
            @test isnothing(model_logistic_gd) == false
            @test length(ŷ_logistic_gd) == size(test.X,1);
            @test isnothing(CM_logistic_gd) == false
            @test size(CM_logistic_gd,1) == 2
            @test size(CM_logistic_gd,2) == 2
            @test all(CM_logistic_gd .>= 0)
            @test sum(CM_logistic_gd) == length(y_logistic_gd)
            @test (CM_logistic_gd[1,1] + CM_logistic_gd[2,1]) > 0
            @test (CM_logistic_gd[1,1] + CM_logistic_gd[1,2]) > 0
            @test 0.0 <= accuracy_logistic <= 1.0
            @test 0.0 <= precision_logistic <= 1.0
            @test 0.0 <= recall_logistic <= 1.0
        end

        @testset "Task 2: Perceptron" begin
            @test isnothing(model_perceptron) == false
            @test length(ŷ_perceptron) == size(test.X,1);
            @test isnothing(CM_perceptron) == false
            @test size(CM_perceptron,1) == 2
            @test size(CM_perceptron,2) == 2
            @test all(CM_perceptron .>= 0)
            @test sum(CM_perceptron) == length(y_perceptron)
            @test (CM_perceptron[1,1] + CM_perceptron[2,1]) > 0
            @test (CM_perceptron[1,1] + CM_perceptron[1,2]) > 0
            @test 0.0 <= accuracy_perceptron <= 1.0
            @test 0.0 <= precision_perceptron <= 1.0
            @test 0.0 <= recall_perceptron <= 1.0
        end

        @testset "Discussion questions" begin
            @test do_I_see_confusion_table == true
            @test do_I_see_parameter_table == true
            @test did_I_answer_DQ1 == true
            @test did_I_answer_DQ2 == true
            @test did_I_answer_DQ3 == true
            @test did_I_answer_DQ4 == true
        end
    end
end;

Setup, Prerequisites and Data: Test Failed at c:\Users\evan\OneDrive\Documents\GitHub\ps2-5820-s26-EvoTW-source\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y130sZmlsZQ==.jl:6
  Expression: isnothing(X) == false
   Evaluated: true == false

Stacktrace:
 [1] top-level scope
   @ c:\Users\evan\OneDrive\Documents\GitHub\ps2-5820-s26-EvoTW-source\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y130sZmlsZQ==.jl:4
 [2] macro expansion
   @ C:\Users\evan\.julia\juliaup\julia-1.12.4+0.x64.w64.mingw32\share\julia\stdlib\v1.12\Test\src\Test.jl:1776 [inlined]
 [3] macro expansion
   @ c:\Users\evan\OneDrive\Documents\GitHub\ps2-5820-s26-EvoTW-source\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y130sZmlsZQ==.jl:5 [inlined]
 [4] macro expansion
   @ C:\Users\evan\.julia\juliaup\julia-1.12.4+0.x64.w64.mingw32\share\julia\stdlib\v1.12\Test\src\Test.jl:1776 [inlined]
 [5] macro expansion
   @ c:\Users\evan\OneDrive\Documents\GitHub\ps2-5820-s26-EvoTW-source\jl_notebook_cell_df34fa98e69747e

Test.TestSetException: Some tests did not pass: 1 passed, 16 failed, 33 errored, 0 broken.